#### The purpose of this notebook is to parse the xls_dist_info_X.csv files created by AnalysisTrajectories from the stat_X.out files.  

In [1]:
import pandas as pd
from pathlib import Path
import multiprocessing


In [2]:
def tuple_to_string(entry):
    return entry[0] + "_" + str(entry[1]) + "_" + entry[2] + "_" + str(entry[3])


def string_to_tuple(entry):
    comps = entry.split("_")
    return comps[0], int(comps[1]), comps[2], int(comps[3])


In [3]:
def get_columns(df, prot_1, res_1, prot_2, res_2):
    matched_col_ids = list()
    for i in range(1, len(df.columns)):
        curr_col = df.columns[i]
        curr_entry = curr_col.split('|')
        curr_prot_1 = curr_entry[3].split('.')[0]
        curr_prot_2 = curr_entry[5].split('.')[0]
        curr_res_1 = int(curr_entry[4])
        curr_res_2 = int(curr_entry[6])

        if prot_1 == curr_prot_1 and prot_2 == curr_prot_2 and res_1 == curr_res_1 and res_2 == curr_res_2:
            matched_col_ids.append(i)
        elif prot_2 == curr_prot_1 and prot_1 == curr_prot_2 and res_2 == curr_res_1 and res_1 == curr_res_2:
            matched_col_ids.append(i)

    return matched_col_ids

In [4]:
scores_file = Path("/Users/matthew/Documents/mtorc2/single_traj_experiments/exp_5/37/analysis/traj/scores_info_0.csv")
scores_df = pd.read_csv(scores_file)
scores_df[scores_df["Total_Score"] == scores_df["Total_Score"].min()].index[0]



7295

In [5]:
file = Path("/Users/matthew/Documents/mtorc2/single_traj_experiments/exp_5/35/analysis/traj/XLs_dist_info_0.csv")
df = pd.read_csv(file)
df = df.drop(columns=["Unnamed: 0"])
df.head()

init_file = Path("/Users/matthew/Documents/mtorc2/data/xlms/monomer.xl_satisfaction.csv")
init_df = pd.read_csv(init_file)
init_df = init_df.drop(columns=["Unnamed: 0"])
init_df.satisfied = False
init_df.head()


,prot1,res1,prot2,res2,contains,satisfied
0,MTOR,1197,RICTOR,516,False,False
1,MTOR,1218,RICTOR,516,False,False
2,RICTOR,1642,MSIN1,102,False,False
3,RICTOR,1092,MTOR,1993,False,False
4,RICTOR,1642,MSIN1,104,False,False


In [27]:
meta_df = init_df.copy()
meta_df["copy1"] = 0 
meta_df["copy2"] = 0 
meta_df.head()

,prot1,res1,prot2,res2,contains,satisfied,copy1,copy2
0,MTOR,1197,RICTOR,516,False,False,0,0
1,MTOR,1218,RICTOR,516,False,False,0,0
2,RICTOR,1642,MSIN1,102,False,False,0,0
3,RICTOR,1092,MTOR,1993,False,False,0,0
4,RICTOR,1642,MSIN1,104,False,False,0,0


In [6]:
df.head()

,MC_frame,CrossLinkingMassSpectrometryRestraint_Distance_|DSS|0.1|MTOR.0|1197|RICTOR.0|516|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|DSS|0.1|MTOR.0|379|MTOR.0|251|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|DSS|0.1|RICTOR.0|516|RICTOR.0|507|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|DSS|0.2|MTOR.0|1197|RICTOR.1|516|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|DSS|0.2|MTOR.1|379|MTOR.0|251|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|DSS|0.2|RICTOR.1|516|RICTOR.0|507|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|DSS|0.3|MTOR.0|379|MTOR.1|251|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|DSS|0.3|MTOR.1|1197|RICTOR.0|516|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|DSS|0.3|RICTOR.0|516|RICTOR.1|507|0|PSI|,...,CrossLinkingMassSpectrometryRestraint_Distance_|EDC|5.3|MTOR.1|1218|RICTOR.0|517|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|EDC|5.4|MTOR.1|1218|RICTOR.1|517|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|EDC|7.1|MTOR.0|1458|RICTOR.0|1080|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|EDC|7.2|MTOR.0|1458|RICTOR.1|1080|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|EDC|7.3|MTOR.1|1458|RICTOR.0|1080|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|EDC|7.4|MTOR.1|1458|RICTOR.1|1080|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|EDC|9.1|MTOR.0|1457|RICTOR.0|1095|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|EDC|9.2|MTOR.0|1457|RICTOR.1|1095|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|EDC|9.3|MTOR.1|1457|RICTOR.0|1095|0|PSI|,CrossLinkingMassSpectrometryRestraint_Distance_|EDC|9.4|MTOR.1|1457|RICTOR.1|1095|0|PSI|
0,0.0,30.983407,36.997496,3.368434,70.487238,170.833555,93.038048,170.833555,70.487238,93.038048,...,79.566410,15.275671,113.188596,118.027617,118.027617,113.188596,113.053954,118.155857,118.155857,113.053954
1,1.0,32.509803,40.258970,5.697732,71.577730,169.410987,94.025174,169.410987,71.577730,94.025174,...,80.206202,16.606449,110.132776,114.429411,114.429411,110.132776,111.933164,117.775425,117.775425,111.933164
2,2.0,32.997982,42.894682,9.007608,71.897098,171.015527,93.991099,171.015527,71.897098,93.991099,...,79.841890,18.022571,108.535290,115.687791,115.687791,108.535290,110.338821,117.680249,117.680249,110.338821
3,3.0,33.343479,46.118494,12.901072,72.137405,171.496858,94.244746,171.496858,72.137405,94.244746,...,79.893433,18.552402,107.330446,114.131460,114.131460,107.330446,108.495685,116.720780,116.720780,108.495685
4,4.0,30.898295,48.531366,12.719099,70.100079,171.254018,93.120174,171.254018,70.100079,93.120174,...,78.274713,16.374432,105.068381,113.715537,113.715537,105.068381,105.798748,115.739020,115.739020,105.798748


In [17]:
col_strs = list()
copy_orders_dict = dict() 
matched_cols_dict = dict() 
for i in range(len(init_df)):
    prot1, res1, prot2, res2 = tuple(init_df.iloc[i, 0:4])
    entry = (prot1, res1, prot2, res2)
    
    copy_orders_dict[entry] = list() 
    matched_cols = get_columns(df, prot1, res1, prot2, res2)
    if len(matched_cols) > 0:
        col_strs.append(tuple_to_string(entry))
        matched_cols_dict[entry] = matched_cols
        for col in matched_cols: 
            copy_1 = df.columns[col].split('|')[3].split(".")[1]
            copy_2 = df.columns[col].split('|')[5].split(".")[1]
            copy_orders_dict[entry].append((copy_1, copy_2))
            


In [18]:
copy_orders_dict

{('MTOR', 1197, 'RICTOR', 516): [('0', '0'),
  ('0', '1'),
  ('1', '0'),
  ('1', '1')],
 ('MTOR', 1218, 'RICTOR', 516): [('0', '0'),
  ('0', '1'),
  ('1', '0'),
  ('1', '1')],
 ('RICTOR', 1642, 'MSIN1', 102): [],
 ('RICTOR', 1092, 'MTOR', 1993): [('0', '0'),
  ('1', '0'),
  ('0', '1'),
  ('1', '1')],
 ('RICTOR', 1642, 'MSIN1', 104): [],
 ('MTOR', 1186, 'RICTOR', 516): [('0', '0'),
  ('0', '1'),
  ('1', '0'),
  ('1', '1')],
 ('AKT1', 30, 'RICTOR', 11): [],
 ('RICTOR', 1477, 'MTOR', 2045): [('0', '0'),
  ('1', '0'),
  ('0', '1'),
  ('1', '1')],
 ('MTOR', 1186, 'RICTOR', 507): [('0', '0'),
  ('0', '1'),
  ('1', '0'),
  ('1', '1')],
 ('MTOR', 1218, 'RICTOR', 507): [('0', '0'),
  ('0', '1'),
  ('1', '0'),
  ('1', '1')],
 ('MSIN1', 422, 'MLST8', 215): [],
 ('AKT1', 30, 'RICTOR', 813): [],
 ('MTOR', 42, 'RICTOR', 1382): [('0', '0'),
  ('0', '1'),
  ('1', '0'),
  ('1', '1')],
 ('MSIN1', 95, 'RICTOR', 1614): [],
 ('RICTOR', 1477, 'MTOR', 2090): [('0', '0'),
  ('1', '0'),
  ('0', '1'),
  ('1', '

In [19]:
matched_cols_dict

{('MTOR', 1197, 'RICTOR', 516): [1, 4, 8, 10],
 ('MTOR', 1218, 'RICTOR', 516): [13, 16, 20, 22],
 ('RICTOR', 1092, 'MTOR', 1993): [146, 149, 152, 155],
 ('MTOR', 1186, 'RICTOR', 516): [165, 168, 172, 174],
 ('RICTOR', 1477, 'MTOR', 2045): [186, 189, 192, 195],
 ('MTOR', 1186, 'RICTOR', 507): [197, 200, 204, 206],
 ('MTOR', 1218, 'RICTOR', 507): [209, 212, 216, 218],
 ('MTOR', 42, 'RICTOR', 1382): [42, 44, 48, 51],
 ('RICTOR', 1477, 'MTOR', 2090): [63, 66, 69, 72],
 ('MTOR', 550, 'RICTOR', 516): [74, 76, 80, 83],
 ('RICTOR', 1391, 'MTOR', 1566): [95, 98, 101, 104],
 ('MTOR', 2129, 'RICTOR', 1312): [137, 139, 142, 143],
 ('MTOR', 1458, 'RICTOR', 1119): [221, 222, 223, 224],
 ('MTOR', 1458, 'RICTOR', 1082): [237, 238, 239, 240],
 ('MTOR', 1458, 'RICTOR', 1095): [241, 242, 243, 244],
 ('MTOR', 1218, 'RICTOR', 517): [245, 246, 247, 248],
 ('MTOR', 1458, 'RICTOR', 1080): [249, 250, 251, 252],
 ('MTOR', 1457, 'RICTOR', 1095): [253, 254, 255, 256],
 ('MTOR', 1458, 'RICTOR', 1107): [225, 226, 2

In [25]:
xl_sat_dict = dict()
for entry in matched_cols_dict.keys(): 
    xl_sat_dict[tuple_to_string(entry)] = list() 

meta_dict = dict() 

for i in range(len(df)):
    for col_str in col_strs:
        entry = string_to_tuple(col_str)
        col_ids = matched_cols_dict[entry]
        copy_orders = copy_orders_dict[entry]
        min_xl_dist = df.iloc[i, col_ids[0]]
        min_copy_id = ('0','0')
        for j in range(4):
            xl_dist = df.iloc[i, col_ids[j]]
            if xl_dist < min_xl_dist:
                min_xl_dist = xl_dist
                min_copy_id = copy_orders[j]
                
        meta_dict[entry] = (min_xl_dist, min_copy_id)
                
        # Update the meta_data 
        xl_sat_dict[col_str].append(min_xl_dist)
    

In [31]:
for entry in meta_dict.keys(): 
    for i in range(len(meta_df)): 
        if entry == tuple(meta_df.iloc[i, 0:4]): 
            meta_df.iloc[i, meta_df.columns.get_loc("contains")] = True
            
            xl_dist, copy_id = meta_dict[entry]
            if xl_dist < 35: 
                meta_df.iloc[i, meta_df.columns.get_loc("satisfied")] = True 
            
            meta_df.iloc[i, meta_df.columns.get_loc("copy1")] = int(copy_id[0]) 
            meta_df.iloc[i, meta_df.columns.get_loc("copy2")] = int(copy_id[1]) 


In [34]:
meta_df.head(50)

,prot1,res1,prot2,res2,contains,satisfied,copy1,copy2
0,MTOR,1197,RICTOR,516,True,True,0,0
1,MTOR,1218,RICTOR,516,True,True,0,0
2,RICTOR,1642,MSIN1,102,False,False,0,0
3,RICTOR,1092,MTOR,1993,True,False,0,0
4,RICTOR,1642,MSIN1,104,False,False,0,0
5,MTOR,1186,RICTOR,516,True,True,0,0
6,AKT1,30,RICTOR,11,False,False,0,0
7,RICTOR,1477,MTOR,2045,True,True,0,0
8,MTOR,1186,RICTOR,507,True,True,0,0
9,MTOR,1218,RICTOR,507,True,True,0,0
